# TODO scrape any car data from ss.com
* clean up prices as numerics
* group by make and model
* plot the data - pricing for sure
* if you do more advanced analysis that also could be used for the final project

* for those who want scraping part of final project
* scrape data from somewhere else - maybe Lithuanian ss.com or wikipedia?

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# url = "https://www.ss.com/en/electronics/phones/mobile-phones/apple/sell/riga_f/"
# url


200

In [ ]:
# req = requests.get(url)  #extracting the html
# req.status_code

In [4]:
soup = BeautifulSoup(req.text, 'lxml') 
soup.title

<title>SS.COM Mobile phones - Apple, Prices, Sell - Advertisements. Riga</title>

In [5]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"}) #this will work as long as html has a table row with id head_line
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]]  # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [6]:
column_names = getColList(soup)
column_names  # to be used later when creating our table

['description', 'url', 'Model', 'Volume, Gb', 'Cond.', 'Price']

In [7]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [8]:
# from row get dictionary with values 
def getRow(row, colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [9]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [10]:
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows)
    

In [11]:
audi_cars = "https://www.ss.com/en/transport/cars/audi/sell/"
idf = getDFfromURL(audi_cars)
idf.head()

,description,url,Model,Year,Volume,Run,Price
0,"Pārdodu Audi kungu, kurš piedzimis iepriekšējā...",https://ss.com/msg/en/transport/cars/audi/a3/b...,A3,1997,1.6,220 thd.,489 €
1,"Auto ir apbruzats laikā gaitā, bet kalpo labi....",https://ss.com/msg/en/transport/cars/audi/a4/b...,A4,1996,1.9D,605 thd.,"1,100 €"
2,В хорошем состоянии,https://ss.com/msg/en/transport/cars/audi/100/...,100,1994,2.0,326 thd.,"1,100 €"
3,"Pārdod audi a3 ar nesen izietu skati, auto lab...",https://ss.com/msg/en/transport/cars/audi/a3/c...,A3,2007,1.9D,334 thd.,"4,800 €"
4,Tikko no Vācijas. \r\nAuto tehniski un vizuāli...,https://ss.com/msg/en/transport/cars/audi/a4/b...,A4,1995,2.6,209 thd.,"5,200 €"


In [12]:
idf.shape()

TypeError: 'tuple' object is not callable